# Đề thi mẫu chứng chỉ TensorFlow - Dạng bài 04

```
ProtonX - TensorFlow Class
```
Giới thiệu về chứng chỉ - https://www.tensorflow.org/certificate

> Chú ý: đọc kỹ các nội dung trong Handle Notebook trong liên kết trên.

---

**Bài toán: Phân loại câu mỉa mai !**

### Hướng dẫn làm bài  
- Trong phần bài tập này, các bạn sẽ thực hiện tương tự bài IMDB Dataset.

Yêu cầu:
- Sử dụng TensorFlow.
- Sử dụng Callback.
- Sử dụng các mạng LSTM, GRU, Bi-LSTM, v.v...
- Tỷ lệ chính xác trên tập Test > 85 - 87%.



### Thực hiện

Dataset: https://rishabhmisra.github.io/publications/

In [ ]:
# Hướng dẫn về bộ dữ liệu này được thực hiện trong Lab Tokenizer
!wget --no-check-certificate \
    https://storage.googleapis.com/learning-datasets/sarcasm.json \
    -O /tmp/sarcasm.json

import json

with open("/tmp/sarcasm.json", 'r') as f:
    datastore = json.load(f)

--2023-10-10 16:59:10--  https://storage.googleapis.com/learning-datasets/sarcasm.json
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.207, 173.194.74.207, 173.194.192.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5643545 (5.4M) [application/json]
Saving to: ‘/tmp/sarcasm.json’

/tmp/sarcasm.json   100%[===================>]   5.38M  --.-KB/s    in 0.06s   

2023-10-10 16:59:10 (97.6 MB/s) - ‘/tmp/sarcasm.json’ saved [5643545/5643545]



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.models import Sequential

In [ ]:
import numpy as np

dataset = []
label_dataset = []
for item in datastore:
    dataset.append(item["headline"])
    label_dataset.append(item["is_sarcastic"])

dataset = np.array(dataset)
label_dataset = np.array(label_dataset)
train_size = 0.8
size = int(len(dataset) * train_size)

train_sentence = dataset[:size]
test_sentence = dataset[size:]

train_label = label_dataset[:size]
test_label = label_dataset[size:]

In [ ]:
vocab_size = len(train_sentence)
embedding_size = 64
max_length = 25

tokenizer = Tokenizer(num_words=vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(train_sentence)

train_sequences = tokenizer.texts_to_sequences(train_sentence)
padded_train_sequences = pad_sequences(train_sequences, maxlen=max_length, truncating="post", padding="post")

test_sequences = tokenizer.texts_to_sequences(test_sentence)
padded_test_sequences = pad_sequences(test_sequences, maxlen=max_length, truncating="post", padding="post")

In [ ]:
units = 128
embedding_size = 100
vocab_size = len(tokenizer.index_word) + 1
input_length = max_length

In [ ]:
model_lstm = tf.keras.Sequential([
  tf.keras.layers.Embedding(vocab_size, 64),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])
#https://www.kaggle.com/code/nikhilkohli/sarcasm-detection-using-lstm-gru-85-accuracy

In [ ]:
rlrp = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2)
model_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
model_lstm.fit(
    padded_train_sequences, train_label,
    batch_size=32,
    epochs=100,
    validation_data=(padded_test_sequences, test_label),
    callbacks=[rlrp] ,
    verbose=1)

Epoch 1/100
668/668 [==============================] - 56s 33ms/step - loss: 0.3892 - acc: 0.8121 - val_loss: 0.3295 - val_acc: 0.8598 - lr: 0.0010
Epoch 2/100
668/668 [==============================] - 10s 15ms/step - loss: 0.1711 - acc: 0.9341 - val_loss: 0.3576 - val_acc: 0.8560 - lr: 0.0010
Epoch 3/100
668/668 [==============================] - 9s 14ms/step - loss: 0.0715 - acc: 0.9752 - val_loss: 0.4665 - val_acc: 0.8501 - lr: 0.0010
Epoch 4/100
668/668 [==============================] - 9s 13ms/step - loss: 0.0235 - acc: 0.9927 - val_loss: 0.5828 - val_acc: 0.8529 - lr: 1.0000e-04
Epoch 5/100
668/668 [==============================] - 10s 15ms/step - loss: 0.0135 - acc: 0.9963 - val_loss: 0.6553 - val_acc: 0.8532 - lr: 1.0000e-04
Epoch 6/100
668/668 [==============================] - 10s 14ms/step - loss: 0.0086 - acc: 0.9980 - val_loss: 0.6711 - val_acc: 0.8521 - lr: 1.0000e-05
Epoch 7/100
668/668 [==============================] - 9s 13ms/step - loss: 0.0081 - acc: 0.9984 - val

In [ ]:
# Lưu Model và nộp
model_lstm.save("sarcasm_model_lstm.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
